In [1]:
import os
os.chdir("/home/david/Documents/projects/app-local-rag-repo/")

In [5]:
import ast
from collections import defaultdict
import os

class DependencyAnalyzer(ast.NodeVisitor):
    def __init__(self, module_path):
        self.imports = {}
        self.local_defs = set()
        self.dep_tree = defaultdict(list)
        self.current_scope = None
        self.scope_stack = []
        self.module_path = module_path
        self.variables = defaultdict(dict)
        # Track methods of classes: {class_name: set(method_names)}
        self.class_methods = defaultdict(set)
        # Track current class context
        self.current_class = None

    def visit_ClassDef(self, node):
        self.local_defs.add(node.name)
        old_class = self.current_class
        self.current_class = node.name
        self.scope_stack.append(node.name)
        self.generic_visit(node)
        self.scope_stack.pop()
        self.current_class = old_class

    def visit_FunctionDef(self, node):
        self.local_defs.add(node.name)
        # If inside a class, register this function as a method.
        if self.current_class is not None:
            self.class_methods[self.current_class].add(node.name)
        self.scope_stack.append(node.name)
        self.current_scope = node.name
        self.generic_visit(node)
        self.scope_stack.pop()
        self.current_scope = self.scope_stack[-1] if self.scope_stack else None

    def _resolve_call(self, node):
        """Resolve the full name of a call with consistent :: notation."""
        if isinstance(node.func, ast.Name):
            name = node.func.id
            if name in self.imports and not name.startswith(('os', 'sys')):
                full_path = self.imports[name]
                if '.' in full_path:
                    module_path, class_name = full_path.rsplit('.', 1)
                    return f"{module_path}::{class_name}"
                return full_path
            if name in self.local_defs:
                return f"{self.module_path}::{name}"
            return None

        elif isinstance(node.func, ast.Attribute):
            parts = []
            current = node.func
            while isinstance(current, ast.Attribute):
                parts.append(current.attr)
                current = current.value
            if isinstance(current, ast.Name):
                base = current.id
                parts.append(base)
                method_or_class = parts[0]

                # Handle self.method calls correctly:
                if base == 'self' and self.current_class is not None:
                    if method_or_class in self.class_methods.get(self.current_class, set()):
                        return f"{self.module_path}::{self.current_class}::{method_or_class}"

                # ... (rest of the original logic unchanged)
                if self.current_scope and base in self.variables.get(self.current_scope, {}):
                    class_path = self.variables[self.current_scope][base]
                    return f"{class_path}::{method_or_class}"

                full_name = '.'.join(reversed(parts))
                if base in self.imports and not base.startswith(('os', 'sys')):
                    imp_full = self.imports[base]
                    if '.' in imp_full:
                        module_path = imp_full.rsplit('.', 1)[0]
                        class_name = imp_full.split('.')[-1]
                        if len(parts) > 1:
                            return f"{module_path}::{class_name}::{method_or_class}"
                        return f"{module_path}::{class_name}"
                if base in self.local_defs:
                    return f"{self.module_path}::{full_name}"
                return None
        return None

    def visit_Lambda(self, node):
        if self.current_scope:
            self.generic_visit(node)

    def get_dependency_tree(self):
        # Remove duplicates while preserving the order of appearance.
        def unique_order(seq):
            seen = set()
            result = []
            for item in seq:
                if item not in seen:
                    seen.add(item)
                    result.append(item)
            return result
        return {k: unique_order(v) for k, v in self.dep_tree.items()}

def analyze_script(script_path):
    module_path = script_path.replace(os.sep, '.').replace('.py', '')
    with open(script_path, 'r') as f:
        script_content = f.read()
    tree = ast.parse(script_content)
    analyzer = DependencyAnalyzer(module_path)
    analyzer.visit(tree)
    return analyzer.get_dependency_tree()

# Example usage
if __name__ == "__main__":
    # Replace with your script path
    global_path = "/home/david/Documents/glovo/machine-learning-platform/"
    script_path = "widget_framework/src/widget_builder.py"
    filepath = os.path.join(global_path, script_path)
    dep_tree = analyze_script(filepath)
    for func, deps in dep_tree.items():
        print(f"{func}:")
        for dep in deps:
            print(f"  - {dep}")
